# 村田安雄『動的経済システムの最適制御』第3章2節bの検算

(Version: 0.0.2)

「第3章 離散時間ダイナミック・プログラミングの方法と消費計画への応用」の「2. 確実性下の D.P. の最適化原理」の「［b］m 制御変数と n 状態変数の D.P.」について、行列に関する計算の「検算」を行ってみる。

これは sympy_matrix_tools のテストでもある。

本当はこういった証明は専門の theorem prover を使ってやるべきである。しかし、無料で誰でも使える Google Colab & Python で示せればそれはそれで価値があるのではないかと考えた。

In [1]:
from time import gmtime, strftime
print("Time-stamp: <%s>" % strftime("%Y-%m-%dT%H:%M:%SZ", gmtime()))

Time-stamp: <2022-05-16T10:38:13Z>


In [2]:
!pip install git+https://github.com/JRF-2018/sympy_matrix_tools

  Cloning https://github.com/JRF-2018/sympy_matrix_tools to /tmp/pip-req-build-fkqemajz
  Running command git clone -q https://github.com/JRF-2018/sympy_matrix_tools /tmp/pip-req-build-fkqemajz


In [3]:
import sympy
sympy.__version__

'1.7.1'

In [4]:
import sympy_matrix_tools
sympy_matrix_tools.__version__

'0.1.8'

In [5]:
from sympy import *
init_printing()
from sympy_matrix_tools import *

In [6]:
t = Symbol("t", integer=True, positive=True)
n = Symbol("n", integer=True, positive=True)
m = Symbol("m", integer=True, positive=True)
y = MatrixFunction("y", n, 1)
x = MatrixFunction("x", m, 1)
z = MatrixFunction("z", n, 1)
A = MatrixSymbol("A", n, n)
B = MatrixSymbol("B", n, m)

EQ11 = Eq(y(t), A * y(t -1) + B * x(t) + z(t))

なお P, Q はのちに定義する P, Q の MatrixFunction と名前がダブるため P_T, Q_T と名付けておく。 

In [7]:
T = Symbol("T", integer=True, positive=True)
tau = Symbol("tau", integer=True, positive=True)
beta = Symbol("beta", real=True)
k = Symbol("k", integer=True, positive=True)

P_T = MatrixSymbol("P_T", n, 1)
Q_T = MatrixSymbol("Q_T", n, n)

EQQTT = Eq(Q_T.T, Q_T)

J = MatrixFunction("J", 1, 1)
w = MatrixFunction("w", 1, 1)

EQ12 = Eq(J(T, 1), Sum((beta ** (t - 1)) * w(y(t)), (t, 1, T)))
EQ13 = Eq(w(y(t)), P_T.T * y(t) + Rational(1/2) * y(t).T * Q_T * y(t))
EQ14 = Eq(y(t), A**(t - (tau - 1)) * y(tau - 1)
          + MatSum(A ** (t - k) * (B * x(k) ++ z(k)), (k, tau, t)))

ここで x(tau) の系列に関して式(15)や式(16)を定義するのは面倒なので、直接、式(17)を天下り的に次のように定義したい。しかし、SymPy の Min はその引数に集合をとることができない。

In [8]:
# v = MatrixFunction("v", 1, 1)
# x_tau = Symbol("x_tau", real=True)

# EQ17 = Eq(v(tau, y(tau - 1)),
#           Min(ImageSet(Lambda(x_tau, Subs(w(y(tau)) + beta * v(tau + 1, y(tau)),
#                                    x(tau), x_tau)), Interval(0, oo))))

そこで定義できたとしてもやるべきことだった微分のための式のみ定義しておく。

In [9]:
v = MatrixFunction("v", 1, 1)

EQ17_lhs = v(tau, y(tau - 1))
EQ17_rhs = w(y(tau)) + beta * v(tau + 1, y(tau))

さてここから本では明示してないが、帰納法で式を示すことになる。

In [10]:
S = MatrixFunction("S", n, n)
P = MatrixFunction("P", n, 1)
Q = MatrixFunction("Q", n, n)
h = MatrixFunction("h", 1, 1)
g = MatrixFunction("g", 1, 1)

EQ21D = Eq(S(t), Identity(n) - B * ((B.T * Q(t) * B) ** - 1) * B.T * Q(t))
EQ24D = Eq(Q(t), Q_T + beta * A.T * Q(t + 1) * S(t + 1) * A)
EQ23D = Eq(P(t), P_T + beta * A.T * S(t + 1).T * P(t + 1)
           + beta * A.T * Q(t + 1) * S(t + 1) * z(t + 1))
EQ22D = Eq(h(t), P(t).T * B * ((B.T * Q(t) * B) ** -1) * B.T * P(t))
EQ25D = Eq(g(t), beta * (P(t + 1).T * S(t + 1) * z(t + 1)
                         + (Rational(1, 2) * z(t + 1).T * Q(t + 1) * S(t + 1)
                            * z(t + 1))
                         - Rational(1, 2) * h(t + 1) + g(t + 1)))
EQGT = Eq(g(T), ZeroMatrix(1, 1))

EQ19D = Eq(x(t), - ((B.T * Q(t) * B) ** -1) * B.T *
           (Q(t) * (A * y(t - 1) + z(t)) + P(t)))

EQ20D = Eq(v(t, y(t - 1)),
           (P(t).T + Rational(1, 2) * (A * y(t -1) + z(t)).T * Q(t))
           * S(t) * (A * y(t -1) + z(t))
           - Rational(1, 2) * h(t) + g(t))

示したいのは t := t において EQ21D～EQ25D のときに EQ19D EQ20D が成り立っていたとき、t := t-1 において、EQ21D～EQ25D と定義したとき EQ19D EQ20D が成り立つことである。また帰納法の初期条件として t := T-1 において EQ21D～EQ25D と定義したとき EQ19D EQ20D が成り立つこともまず示さねばならない。

まずは本にしたがい、t := T-1 の場合を示す。

まず次を x(t) で微分したい。

In [11]:
EQ18a_rhs = P_T.T * y(T) + Rational(1, 2) * y(T).T * Q_T * y(T)

In [12]:
EQ11tmp = EQ11.subs(t, T)
EQ18tmp = EQ18a_rhs.subs(EQ11tmp.lhs, EQ11tmp.rhs)
EQ18tmp

MatrixFunction はまだ微分に対応していないのでいったんここに出てくるものを MatrixSymbol に置き換えねばならない。

In [13]:
z_T = MatrixSymbol("z_T", n, 1)
y_Tm1 = MatrixSymbol("y_Tm1", n, 1)
x_T = MatrixSymbol("x_T", m, 1)
EQ18tmp2 = EQ18tmp.subs({x(T): x_T, y(T - 1): y_Tm1, z(T): z_T})
EQ18tmp2

In [14]:
EQ18tmp3 = EQ18tmp2.diff(x_T)\
                   .subs(EQQTT.lhs, EQQTT.rhs)\
                   .subs({x_T: x(T), y_Tm1: y(T - 1), z_T: z(T)})\
                   .doit()
EQ18tmp3

In [15]:
EQ18tmp4 = EQ18tmp3.expand().doit()
EQ18tmp4

In [16]:
EQ18tmp5 = mat_collect(mat_divide(EQ18tmp4, B.T * Q_T * B), ((B.T * Q_T * B) ** -1) * B.T, right=True)
EQ18tmp5

EQ18tmp5 == 0 と置いて、これが EQ19D と同じことを示したい。

In [17]:
EQ19a = EQ19D.subs(t, T).subs(Q(T), Q_T).subs(P(T), P_T)
(EQ19a.lhs - EQ19a.rhs - EQ18tmp5).expand().doit()

E19a と EQ18tmp5 は同じことであることが示せた。最適値を求めるためにこれを EQ18tmp に代入して整理していく。

In [18]:
EQ20tmp = EQ18tmp.subs(EQ19a.lhs, EQ19a.rhs)
EQ20tmp

In [19]:
EQ20tmp2 = mat_trivial_divide(EQ20tmp.expand().subs(EQQTT.lhs, EQQTT.rhs)).doit()
EQ20tmp2

これが EQ20D と等しいことを示したい。

In [20]:
EQ21a = EQ21D.subs(t, T).subs(Q(T), Q_T)
EQ22a = EQ22D.subs(t, T).subs(Q(T), Q_T).subs(P(T), P_T)
EQ20a = EQ20D.subs(t, T).subs(Q(T), Q_T).subs(P(T), P_T)\
                                        .subs(EQGT.lhs, EQGT.rhs).doit()

EQ20tmp3 = EQ20a.rhs.subs({EQ21a.lhs: EQ21a.rhs, EQ22a.lhs: EQ22a.rhs})\
                    .expand().doit()
EQ20tmp3

In [21]:
(EQ20tmp2 - EQ20tmp3).expand().doit()

これで式(20a) が正しいことが示せた。今後～tmp 系は再利用していくので混乱しないよう。

次に式(18b)を示していこう。

In [22]:
EQ17_lhs_tmp = EQ17_lhs.subs(tau, T - 1)
EQ17_rhs_tmp = EQ17_rhs.subs(tau, T - 1)
EQ13tmp = EQ13.subs(t, T -1)
EQ18tmp = EQ17_rhs_tmp.subs({EQ13tmp.lhs: EQ13tmp.rhs, EQ20a.lhs: EQ20a.rhs})
EQ18tmp

In [23]:
EQ13tmp

In [24]:
EQ18b_rhs = P(T - 1).T * y(T - 1) \
    + Rational(1, 2) * y(T - 1).T * Q(T - 1) * y(T-1) + g(T - 1)
EQ24tmp = EQ24D.subs(t, T - 1)
EQ23tmp = EQ23D.subs(t, T - 1)
EQ25tmp = EQ25D.subs(t, T - 1).subs(EQGT.lhs, EQGT.rhs).doit()
EQ18tmp2 = EQ18b_rhs.subs({EQ24tmp.lhs: EQ24tmp.rhs,
                           EQ23tmp.lhs: EQ23tmp.rhs,
                           EQ25tmp.lhs: EQ25tmp.rhs,
                           Q(T): Q_T,
                           P(T): P_T,
                           EQQTT.lhs: EQQTT.rhs})
EQ18tmp2

In [25]:
EQ21tmp = EQ21D.subs(t, T).subs(Q(T), Q_T)
EQ18tmp3 = (EQ18tmp - EQ18tmp2).subs(EQ21tmp.lhs, EQ21tmp.rhs).expand()\
    .subs(EQQTT.lhs, EQQTT.rhs).doit()
EQ18tmp3

今次の式が成り立つことから、B の逆行列は取れないが、X == Q_T と仮定する。

In [26]:
X = MatrixSymbol("X", n, n)
EQtmp = Eq(X, (Q_T * B * (B.T * Q_T * B) ** -1 * B.T * Q_T))
EQtmp2 = Eq(EQtmp.lhs * B, mat_trivial_divide(EQtmp.rhs * B))
EQtmp2

In [27]:
EQ18tmp3.subs(EQtmp.rhs, Q_T).doit()

ちょっと裏技っぽいのを使ったが、式(18b)は示せたものとする。

In [28]:
EQ11tmp = EQ11.subs(t, T - 1)
EQ18tmp = EQ18b_rhs.subs(EQ11tmp.lhs, EQ11tmp.rhs)
EQ18tmp

In [29]:
z_Tm1 = MatrixSymbol("z_Tm1", n, 1)
y_Tm2 = MatrixSymbol("y_Tm2", n, 1)
x_Tm1 = MatrixSymbol("x_Tm1", m, 1)
P_Tm1 = MatrixSymbol("P_Tm1", n, 1)
Q_Tm1 = MatrixSymbol("Q_Tm1", n, n)
g_Tm1 = MatrixSymbol("g_Tm1", 1, 1)
EQ18tmp2 = EQ18tmp.subs({x(T - 1): x_Tm1,
                         y(T - 2): y_Tm2,
                         z(T - 1): z_Tm1,
                         P(T - 1): P_Tm1,
                         Q(T - 1): Q_Tm1,
                         g(T - 1): g_Tm1})
EQ18tmp3 = EQ18tmp2.diff(x_Tm1)
EQ18tmp4 = EQ18tmp3.subs({x_Tm1: x(T - 1),
                          y_Tm2: y(T - 2),
                          z_Tm1: z(T - 1),
                          P_Tm1: P(T - 1),
                          Q_Tm1: Q(T - 1),
                          g_Tm1: g(T - 1)})
EQ18tmp4

さてここで帰納法で Q(t).T == Q(t) を示す。t := T のときは言えているので、Q(t + 1).T == Q(t + 1) が言えているとして Q(t).T == Q(t) を示す。

In [30]:
EQ21tmp = EQ21D.subs(t, t + 1)
EQ24tmp = EQ24D.subs(EQ21tmp.lhs, EQ21tmp.rhs).expand().doit()

In [31]:
EQ24tmp

In [32]:
EQ24tmp2 = EQ24tmp.rhs.T.subs(Q(t + 1).T, Q(t + 1)).subs(EQQTT.lhs, EQQTT.rhs).doit()
(EQ24tmp.rhs - EQ24tmp2).expand().doit()

ゆえに次が言えた。

In [33]:
EQQtT = Eq(Q(t).T, Q(t))

微分に戻ろう。

In [34]:
EQQtTtmp = EQQtT.subs(t, T - 1)
EQ18tmp5 = EQ18tmp4.subs(EQQtTtmp.lhs, EQQtTtmp.rhs).expand().doit()
EQ18tmp5

In [35]:
EQ18tmp6 = mat_collect(mat_divide(EQ18tmp5, B.T * Q(T - 1) * B), ((B.T * Q(T - 1) * B) ** -1) * B.T, right=True)
EQ18tmp6

EQ18tmp6 == 0 と置いて、これが EQ19D と同じことを示したい。

In [36]:
EQ19b = EQ19D.subs(t, T - 1)
(EQ19b.lhs - EQ19b.rhs - EQ18tmp6).expand().doit()

E19b と EQ18tmp6 は同じことであることが示せた。これを最適値を求めるため EQ18tmp に代入して整理していく。

In [37]:
EQ20tmp = EQ18tmp.subs(EQ19b.lhs, EQ19b.rhs)
EQ20tmp

In [38]:
EQQtTtmp = EQQtT.subs(t, T - 1)
EQ20tmp2 = mat_trivial_divide(EQ20tmp.expand().subs(EQQtTtmp.lhs, EQQtTtmp.rhs)).doit()
EQ20tmp2

これが EQ20D と同じであることを示したい。

In [39]:
EQ21tmp = EQ21D.subs(t, T - 1)
EQ22tmp = EQ22D.subs(t, T - 1)
EQ20tmp = EQ20D.subs(t, T - 1)

EQ20tmp3 = EQ20tmp.rhs.subs({EQ21tmp.lhs: EQ21tmp.rhs,
                             EQ22tmp.lhs: EQ22tmp.rhs})\
                      .expand().doit()
EQ20tmp3

In [40]:
(EQ20tmp2 - EQ20tmp3).expand().doit()

これで t := T - 1 において、EQ19D EQ20D が成り立つことが示せたことになる。

次に帰納法の本題である。t := t において EQ19D EQ20D が成り立っているとき、t := t - 1 において EQ19D EQ20D が成り立っていることを示す。


In [41]:
EQ17_lhs_tmp = EQ17_lhs.subs(tau, t - 1)
EQ17_rhs_tmp = EQ17_rhs.subs(tau, t - 1)
EQ13tmp = EQ13.subs(t, t -1)
EQ18tmp = EQ17_rhs_tmp.subs({EQ13tmp.lhs: EQ13tmp.rhs, EQ20D.lhs: EQ20D.rhs})
EQ18tmp

In [42]:
EQ18c_rhs = P(t - 1).T * y(t - 1) \
    + Rational(1, 2) * y(t - 1).T * Q(t - 1) * y(t-1) + g(t - 1)
EQ24tmp = EQ24D.subs(t, t - 1)
EQ23tmp = EQ23D.subs(t, t - 1)
EQ25tmp = EQ25D.subs(t, t - 1).subs(EQGT.lhs, EQGT.rhs).doit()
EQ18tmp2 = EQ18c_rhs.subs({EQ24tmp.lhs: EQ24tmp.rhs,
                           EQ23tmp.lhs: EQ23tmp.rhs,
                           EQ25tmp.lhs: EQ25tmp.rhs})
EQ18tmp2

In [43]:
EQ18tmp3 = (EQ18tmp - EQ18tmp2)\
    .subs(EQ21D.lhs, EQ21D.rhs).expand()\
    .subs(EQQtT.lhs, EQQtT.rhs).expand().doit()
EQ18tmp3

ここで上の X のハックと同じものが成立しているとする。

In [44]:
EQ18tmp3.subs((Q(t) * B * (B.T * Q(t) * B) ** -1 * B.T * Q(t)), Q(t)).doit()

In [45]:
EQ11tmp = EQ11.subs(t, t - 1)
EQ18tmp = EQ18c_rhs.subs(EQ11tmp.lhs, EQ11tmp.rhs)
EQ18tmp

In [46]:
z_tm1 = MatrixSymbol("z_tm1", n, 1)
y_tm2 = MatrixSymbol("y_tm2", n, 1)
x_tm1 = MatrixSymbol("x_tm1", m, 1)
P_tm1 = MatrixSymbol("P_tm1", n, 1)
Q_tm1 = MatrixSymbol("Q_tm1", n, n)
g_tm1 = MatrixSymbol("g_tm1", 1, 1)
EQ18tmp2 = EQ18tmp.subs({x(t - 1): x_tm1,
                         y(t - 2): y_tm2,
                         z(t - 1): z_tm1,
                         P(t - 1): P_tm1,
                         Q(t - 1): Q_tm1,
                         g(t - 1): g_tm1})
EQ18tmp3 = EQ18tmp2.diff(x_tm1)
EQ18tmp4 = EQ18tmp3.subs({x_tm1: x(t - 1),
                          y_tm2: y(t - 2),
                          z_tm1: z(t - 1),
                          P_tm1: P(t - 1),
                          Q_tm1: Q(t - 1),
                          g_tm1: g(t - 1)})
EQ18tmp4

In [47]:
EQQtTtmp = EQQtT.subs(t, t - 1)
EQ18tmp5 = EQ18tmp4.subs(EQQtTtmp.lhs, EQQtTtmp.rhs).expand().doit()
EQ18tmp5

In [48]:
EQ18tmp6 = mat_collect(mat_divide(EQ18tmp5, B.T * Q(t - 1) * B), ((B.T * Q(t - 1) * B) ** -1) * B.T, right=True)
EQ18tmp6

EQ18tmp6 == 0 と置いて、これが EQ19D と同じことを示したい。

In [49]:
EQ19c = EQ19D.subs(t, t - 1)
(EQ19c.lhs - EQ19c.rhs - EQ18tmp6).expand().doit()

E19c と EQ18tmp6 は同じことであることが示せた。これを最適値を求めるため EQ18tmp に代入して整理していく。

In [50]:
EQ20tmp = EQ18tmp.subs(EQ19c.lhs, EQ19c.rhs)
EQ20tmp

In [51]:
EQQtTtmp = EQQtT.subs(t, t - 1)
EQ20tmp2 = mat_trivial_divide(EQ20tmp.expand().subs(EQQtTtmp.lhs, EQQtTtmp.rhs)).doit()
EQ20tmp2

これが EQ20D と同じことを示したい。

In [52]:
EQ21tmp = EQ21D.subs(t, t - 1)
EQ22tmp = EQ22D.subs(t, t - 1)
EQ20tmp = EQ20D.subs(t, t - 1)

EQ20tmp3 = EQ20tmp.rhs.subs({EQ21tmp.lhs: EQ21tmp.rhs,
                             EQ22tmp.lhs: EQ22tmp.rhs})\
                      .expand().doit()
EQ20tmp3

In [53]:
(EQ20tmp2 - EQ20tmp3).expand().doit()

これで t := t - 1 において、EQ19D EQ20D が成り立つことが示せたことになる。これで帰納法でこの節の導出が正しかったことが証明された。…と思う。

なお、終わってから気づいたのだが、t := T の時点で帰納法を満たしており、t := T - 1 については言う必要がなかったのかもしれない。ただ、本では示しているため、言った意味が全くないとはならないとは思う。